<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/04_Regularization_Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from threading import active_count

from tensorflow.python.keras.layers.core import Activation
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10


# Device setting

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Load data

In [3]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

# Normalize

In [4]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Model

In [5]:
def my_model():
  with tf.device('/device:GPU:0'):
      inputs = keras.Input(shape=(32, 32, 3))
      x = layers.Conv2D(
          32, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(inputs)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(
          64, 3, padding='same', kernel_regularizer=regularizers.l2(0.01) # regularizer 적용
          )(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(
          128, 3, padding='same', kernel_regularizer=regularizers.l2(0.01) # regularizer 적용
          )(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      
      x = layers.Flatten()(x)
      x = layers.Dense(
          64, activation="relu", kernel_regularizer=regularizers.l2(0.01) # regularizer 적용
          )(x)
      x = layers.Dropout(0.5)(x) # Dropout 적용
      outputs = layers.Dense(10)(x)
      model = keras.Model(inputs=inputs, outputs=outputs)
      return model

In [6]:
model = my_model()

In [7]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496 

# Model compile

In [8]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics=['accuracy'],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Train and evaluate

In [9]:
with tf.device('/device:GPU:0'):
  history = model.fit(x_train, y_train, batch_size=64, epochs=150, verbose=2)
  model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/150
782/782 - 19s - loss: 3.0549 - accuracy: 0.3575 - 19s/epoch - 24ms/step
Epoch 2/150
782/782 - 9s - loss: 1.9529 - accuracy: 0.4673 - 9s/epoch - 11ms/step
Epoch 3/150
782/782 - 9s - loss: 1.6501 - accuracy: 0.5156 - 9s/epoch - 11ms/step
Epoch 4/150
782/782 - 9s - loss: 1.5239 - accuracy: 0.5406 - 9s/epoch - 11ms/step
Epoch 5/150
782/782 - 9s - loss: 1.4564 - accuracy: 0.5584 - 9s/epoch - 11ms/step
Epoch 6/150
782/782 - 9s - loss: 1.4120 - accuracy: 0.5722 - 9s/epoch - 11ms/step
Epoch 7/150
782/782 - 9s - loss: 1.3822 - accuracy: 0.5833 - 9s/epoch - 11ms/step
Epoch 8/150
782/782 - 9s - loss: 1.3563 - accuracy: 0.5907 - 9s/epoch - 11ms/step
Epoch 9/150
782/782 - 9s - loss: 1.3374 - accuracy: 0.5973 - 9s/epoch - 11ms/step
Epoch 10/150
782/782 - 9s - loss: 1.3111 - accuracy: 0.6083 - 9s/epoch - 11ms/step
Epoch 11/150
782/782 - 9s - loss: 1.2950 - accuracy: 0.6132 - 9s/epoch - 11ms/step
Epoch 12/150
782/782 - 9s - loss: 1.2820 - accuracy: 0.6188 - 9s/epoch - 11ms/step
Epoch 13/15